Dataset: Visual Pollution Detection  
Model: YOLOv5s  
Author: Shivendra Agarwal  
Connect with me:  
LinkedIn: https://www.linkedin.com/in/shivendra-agarwal-93027a141/    
  
Steps:
1. Download Dataset
2. Convert .csv annotation into Yolov5 readable format
3. Divide into train, valid, test
3. Create a base model with the Test and Valid images for benchmarking, with optimal data, epoch, batch, and image size and save the wieghts
4. Image augmentation using different parameters on train and valid  
  a. Brightness  
  b. Saturation  
  c. Grayscale  
  d. Nosie  
  e. Blur  
  f. Hue(RBG)  
5. Re-train using the older weights with the new set of data (original + augmented) and save.
6. Run detect.py to test the model 

### Import Libraries

In [ ]:
import os
import pandas as pd
from PIL import Image
from imgaug import augmenters as iaa
import numpy as np
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
import shutil
import random
import time

### 1. Download Dataset and view train.csv

In [ ]:
df_antt = pd.read_csv("/content/drive/MyDrive/SA-Garbage Detection/dataset/train.csv")
df_antt.head()

### 2. Convert .csv annotation into Yolov5 readable format

In [ ]:
for index, row in df_antt.iterrows():
  img = row["image_path"]
  ls_bbox = df_antt.loc[df_antt.image_path == img].drop("image_path",axis = 1).apply(lambda x: x.to_dict(),axis = 1).to_list()
  print(ls_bbox)
  img_dim = Image.open(os.path.join("/content/drive/MyDrive/SA-Garbage Detection/dataset/images",img)).size
  ls_out = []
  print(img_dim)
  for bbox in ls_bbox:
    bbox_x = (bbox["xmin"]*2 + bbox["xmax"]*2) / (2 * img_dim[0])
    bbox_y = (bbox["ymin"]*2 + bbox["ymax"]*2) / (2 * img_dim[1])
    bbox_w = (bbox["xmax"]*2 - bbox["xmin"]*2) / img_dim[0]
    bbox_h = (bbox["ymax"]*2 - bbox["ymin"]*2) / img_dim[1]
    class_id = bbox["class"]
    ls_out.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id,bbox_x,bbox_y,bbox_w,bbox_h))

  fl_nm = os.path.join("/content/drive/MyDrive/SA-Garbage Detection/dataset/images",img.replace("jpg","txt"))
  print("\n".join(ls_out),file = open(fl_nm,"w"))

### 3. Divide into train, valid, test

"""   
Train size  = 7874 images  
test size = 2092 images  
total =  9966  
"""

In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/SA-Garbage Detection/dataset/test.csv")
df_test.head()

In [ ]:
# Seperate TEST
for index, row in df_test.iterrows():
  source = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/{row['image_path']}"
  destination = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/test/{row['image_path']}"
  print(source)
  os.rename(source, destination)

In [ ]:
test_img = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/images/test/")
print("test", len(test_img))
train_img = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/images/train/")
print("train", len(train_img))
valid_img = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/images/valid/images")
print("valid", len(valid_img))

In [ ]:
# divide VALID and TRAIN
split_list = []
for iobject in train_img:
  split_list.append(iobject[:-4])
split_list = list(set(split_list))
print(len(split_list))

"""   
train size = 6443  
valid size = 1570 (approx) if we split in train into 80-20  
test size = 2092  
total = 9966  
"""

In [ ]:
valid_img_list = []
for i in range(0, 1570):
  valid_img_list.append(random.choice(split_list))
print(len(valid_img_list))
print(valid_img_list)

In [ ]:
valid_img_list = list(set(valid_img_list))
print(len(valid_img_list))

In [ ]:
for iobject in valid_img_list:
  source = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/train/{iobject}.txt"
  destination = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/valid/{iobject}.txt"
  print(source)
  shutil.move(source, destination)
  source = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/train/{iobject}.jpg"
  destination = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/valid/{iobject}.jpg"
  print(source)
  shutil.move(source, destination)

#### Make directory in YOLOv5 readable format

In [ ]:
test_img = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/images/test/")
print("test", len(test_img))
train_img = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/images/train/")
print("train", len(train_img))
valid_img = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/images/valid/")
print("valid", len(valid_img))

Make two directories (images and labels) in /train/ and /valid/ and move .jpg files in images/ and .txt files in labels/

In [ ]:
# for VALID
for iobject in os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/images/valid"):
  # for images
  if iobject.endswith('.jpg'):
    source = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/valid/{iobject}"
    destination = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/valid/images/{iobject}"
    print(source)
    shutil.move(source, destination)
  # for labels
  elif iobject.endswith('.txt'):
    source = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/valid/{iobject}"
    destination = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/valid/labels/{iobject}"
    print(source)
    shutil.move(source, destination)

In [ ]:
# for TRAIN
for iobject in os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/images/train"):
  # for images
  if iobject.endswith('.jpg'):
    source = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/train/{iobject}"
    destination = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/train/images/{iobject}"
    print(source)
    shutil.move(source, destination)
  # for labels
  elif iobject.endswith('.txt'):
    source = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/train/{iobject}"
    destination = f"/content/drive/MyDrive/SA-Garbage Detection/dataset/images/train/labels/{iobject}"
    print(source)
    shutil.move(source, destination)

#### Make sure that your directory looks like this

Dataset/images/    
  1. train/  
        images/ -> all training images  
        labels/ -> all training labels  

  2. valid/  
        images/ -> all validation images  
        labels/ -> all validation labels  

  3. test/  
        images/ -> all test images  
        
  4. data.yaml -> as format given below

#### data.yaml
train: ../../drive/MyDrive/SA-Garbage Detection/dataset/images/train/images  
val: ../../drive/MyDrive/SA-Garbage Detection/dataset/images/valid/images  
test: ../../drive/MyDrive/SA-Garbage Detection/dataset/images/test/images  

nc: 11  
names: ['GRAFFITI', 'FADED_SIGNAGE', 'POTHOLES', 'GARBAGE', 'CONSTRUCTION_ROAD', 'BROKEN_SIGNAGE', 'BAD_STREETLIGHT', 'BAD_BILLBOARD', 'SAND_ON_ROAD', 'CLUTTER_SIDEWALK', 'UNKEPT_FACADE']

### Create a base model with the Test and Valid images for benchmarking, with optimal data, epoch, batch, and image size and save the wieghts

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-71-gc442a2e Python-3.8.10 torch-1.13.1+cu116 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.6/225.8 GB disk)


In [ ]:
# copy data.yaml to yolov5 folder
!cp "/content/drive/MyDrive/SA-Garbage Detection/dataset/images/data.yaml" /content/yolov5

In [ ]:
# Train YOLOv5s on custom dataset for 30 epochs
!python train.py --img 640 --batch 32 --epochs 30 --data data.yaml --weights yolov5s.pt --cache

In [ ]:
# save weights to drive
!cp -r "/content/yolov5/runs/train/exp" "/content/drive/MyDrive/SA-Garbage Detection"

### Image augmentation (on 50% of dataset) using different parameters on train and valid

In [ ]:
test_img = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/test/images")
print("test", len(test_img))
train_img = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/train/images")
print("train", len(train_img))
valid_img = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/valid/images")
print("valid", len(valid_img))


train_label = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/train/labels")
print("\ntrain", len(train_img))
valid_label = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/valid/labels")
print("valid", len(valid_img))

In [ ]:
def copy(source, destination):
    try:
      shutil.copy(source, destination)
      print("File copied successfully.")
      return True
# If source and destination are same
    except:
      return False

In [ ]:
# Define the directory containing the images
image_dir = "/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/train/images"
label_dir = "/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/train/labels"

aug_image_dir = "/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/train/images"
aug_label_dir = "/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/train/labels"
# Define augmentation sequences
seq = iaa.Sequential([
    iaa.MultiplyBrightness((0.5, 1.5)) #### CHANGE THIS LINE AS PER AUGMENTATION
    ])
count = 0
total_img = len(os.listdir(image_dir))
# Iterate over all image files in the directory
for filename in os.listdir(image_dir):
  if filename.endswith('.jpg'):
      count = count + 1
      print(f"{count}/{total_img} - {filename}")
      # Open image file
      image = Image.open(os.path.join(image_dir, filename))
      # Convert image to numpy array
      image = np.array(image)
      # Determine if image should be augmented (50% chance)
      if np.random.rand() < 0.5:
          # Apply augmentations
          image_aug = seq.augment_image(image)

          # Save augmented image
          Image.fromarray(image_aug).save(os.path.join(aug_image_dir, filename[:-4] + "_aug" + filename[-4:]))

          #copy labels
          aug_filename = f"{filename[:-4]}_aug.txt"

          source = f"{label_dir}/{filename[:-4]}.txt"
          destination = f"{aug_label_dir}/{aug_filename}"

          copy(source, destination)

          # Save original image
          Image.fromarray(image).save(os.path.join(aug_image_dir, filename))
          #copy labels
          aug_filename = f"{filename[:-4]}.txt"
          source = f"{label_dir}/{filename[:-4]}.txt"
          destination = f"{aug_label_dir}/{filename[:-4]}.txt"
          copy(source, destination)


      else:
          # Save original image
          Image.fromarray(image).save(os.path.join(aug_image_dir, filename))
          #copy labels
          aug_filename = f"{filename[:-4]}.txt"
          source = f"{label_dir}/{filename[:-4]}.txt"
          destination = f"{aug_label_dir}/{filename[:-4]}.txt"
          copy(source, destination)

Change the directory path in the above cell and run for valid images

#### Brightness

Define augmentation sequences
```
seq = iaa.Sequential([
    iaa.MultiplyBrightness((0.5, 1.5))
    ])
```



#### Saturation

Define augmentation sequences
```
seq = iaa.Sequential([
    iaa.MultiplySaturation(0.5) # Saturation
    ])
```



#### Grayscale

Define augmentation sequences
```
seq = iaa.Sequential([
    iaa.Grayscale(alpha=1.0)
    ])
```



#### Noise

Define augmentation sequences
```
seq = iaa.Sequential([
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)), # Add gaussian noise with a scale from 0 to 0.05*255 
    ])
```



#### Blur

Define augmentation sequences
```
seq = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0, 2.0)), # add gaussian blur with a random sigma from 0 to 3
    ])
```



#### Hue

Define augmentation sequences  
Red
```
seq = iaa.WithColorspace(to_colorspace="HSV", from_colorspace="RGB", children=iaa.WithChannels(0, iaa.Add((0, 50))))
    ])
```
Green
```
seq = iaa.WithColorspace(to_colorspace="HSV", from_colorspace="RGB", children=iaa.WithChannels(1, iaa.Add((0, 50))))
    ])
```
Blue
```
seq = iaa.WithColorspace(to_colorspace="HSV", from_colorspace="RGB", children=iaa.WithChannels(2, iaa.Add((0, 50))))
    ])
```



Please change line no. 9 in the above code cell to use any of the augmentation technique mentioned above

###### np.random.rand() - to apply only on selected range of images
Augment Technique | train | valid | np.random.rand() <
--- | --- | --- | ---
**Raw** | 6443 | 1431 | 0.5
**Brightness** | 9660 | 2169 |0.5
**Saturation** | 12897 | 2869 | 0.5
**Grayscale** | 16163 | 3618 | 0.5
**Noise** | 19440 | 4340 | 0.5
**Blur** | 22690 | 5060 | 0.5
**Hue-R** | 23277 | 5206 | 0.1
**Hue-G** | 23798 | 5371 | 0.1
**Hue-B** | 24270 | 5499 | 0.1



In [ ]:
test_img = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/test/images")
print("test", len(test_img))
train_img = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/train/images")
print("train", len(train_img))
valid_img = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/valid/images")
print("valid", len(valid_img))


train_label = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/train/labels")
print("\ntrain", len(train_img))
valid_label = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/aug_images/valid/labels")
print("valid", len(valid_img))

### Re-train using the older weights with the new set of data (original + augmented) and save.

#### change dataset path in data.yaml
train: ../../drive/MyDrive/SA-Garbage Detection/dataset/aug_images/train/images  
val: ../../drive/MyDrive/SA-Garbage Detection/dataset/aug_images/valid/images  
test: ../../drive/MyDrive/SA-Garbage Detection/dataset/aug_images/test/images  

nc: 11  
names: ['GRAFFITI', 'FADED_SIGNAGE', 'POTHOLES', 'GARBAGE', 'CONSTRUCTION_ROAD', 'BROKEN_SIGNAGE', 'BAD_STREETLIGHT', 'BAD_BILLBOARD', 'SAND_ON_ROAD', 'CLUTTER_SIDEWALK', 'UNKEPT_FACADE']

In [ ]:
# copy data.yaml to yolov5 folder
!cp "/content/drive/MyDrive/SA-Garbage Detection/dataset/images/data.yaml" /content/yolov5

In [ ]:
# copy previous weight to yolov5 folder
!cp "/content/drive/MyDrive/SA-Garbage Detection/exp5_exp4_batch32_epoch30_img640_aug_yolov5s/weights/best.pt" /content/yolov5

In [ ]:
# Train YOLOv5s on custom dataset for 130 epochs
!python train.py --img 640 --batch 32 --epochs 130 --data data.yaml --weights best.pt --cache

In [ ]:
!cp -r "/content/yolov5/runs/train/exp2" "/content/drive/MyDrive/SA-Garbage Detection"

### Validate the model

In [ ]:
# Validate YOLOv5s on custom dataset
!python val.py --weights "/content/drive/MyDrive/SA-Garbage Detection/exp5_exp4_batch32_epoch30_img640_aug_yolov5s/weights/best.pt" --data data.yaml --img 640 --half

### Detect to test the model

In [ ]:
!python detect.py --save-txt --weights "/content/drive/MyDrive/SA-Garbage Detection/exp5_exp4_batch32_epoch30_img640_aug_yolov5s/weights/best.pt" --img 640 --conf 0.5 --source "/content/drive/MyDrive/SA-Garbage Detection/dataset/images/test/images"
# display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

### Convert yolov5 labels to xmin, ymin, xmax, ymax

In [ ]:
def get_img_labels(img_name, str_v):
  cvmat = cv2.imread("/content/drive/MyDrive/SA-Garbage Detection/dataset/output/" + img_name[:-4] + ".jpg")

  #get height, width
  h,w,_ = cvmat.shape

  #extract x1, y1 <- center, width, height
  class_id = int( float(str_v.split(' ')[0]))
  x1 = int( float(str_v.split(' ')[1]) * w )

  y1 = int( float(str_v.split(' ')[2]) * h )

  xw = int( float(str_v.split(' ')[3]) * w /2)

  yw = int( float(str_v.split(' ')[4]) * h /2)

  #make x1,y1, x2,y2

  start_point = (x1 - xw, y1 - yw )

  end_point   = (x1 + xw, y1 + yw )
  # Define the bounding box coordinates
  xmin = start_point[0]
  ymin = start_point[1]
  xmax = end_point[0]
  ymax = end_point[1]
  return class_id, xmin, ymin, xmax, ymax

In [ ]:
final_test_list = []
class_name = ['GRAFFITI', 'FADED_SIGNAGE', 'POTHOLES', 'GARBAGE', 'CONSTRUCTION_ROAD', 'BROKEN_SIGNAGE', 'BAD_STREETLIGHT', 'BAD_BILLBOARD', 'SAND_ON_ROAD', 'CLUTTER_SIDEWALK', 'UNKEPT_FACADE']
labels_list = os.listdir("/content/drive/MyDrive/SA-Garbage Detection/dataset/output/labels")
for iobject in labels_list:
  with open(os.path.join('/content/drive/MyDrive/SA-Garbage Detection/dataset/output/labels', iobject), 'r') as f:
    labels = f.read()
    print(labels.split('\n'))
    for jobject in labels.split('\n'):
      if len(jobject) > 0:
        class_id, xmin, ymin, xmax, ymax = get_img_labels(iobject, jobject)
        print(class_id, xmin, ymin, xmax, ymax)
        final_test_list.append({"class": class_id, "image_path": iobject[:-4]+".jpg", "name": class_name[class_id], "xmax": xmax, "xmin": xmin, "ymax": ymax, "ymin": ymin})        

In [ ]:
dict_to_df = pd.DataFrame.from_dict(final_test_list)

In [ ]:
dict_to_df.to_csv('/content/drive/MyDrive/SA-Garbage Detection/dataset/test.csv', index=False)